# Estimating the Microcides committed by different types of road user in the UK using STATS19 data 2005-2014 - Data Exploration

This notebook takes traffic volume and accident data from the DfT as with the related Traffic_Microcides notebook. Several different transformations of the data are created and the conclusions as they related to the final settings are stated at the bottom. Most of the code is the same as the Traffic_Microcides notebook. This notebook has more functions to allow multiple different settings to be run easily.

In [1]:
import pandas as pd
import requests, zipfile
import io
from numpy import where
import datetime

In [2]:
"""The links to the STATS19 data and 2 Traffic Volume Files"""

Offline = True

if Offline:
    STATS19 = "http://localhost:8888/files/Stats19_Data_2005-2014.zip"
    Traffic_Volume = "http://localhost:8888/files/tra0201.xlsx"
    Cycle_Volume = "http://localhost:8888/files/tra0401.xlsx"
else:
    STATS19 = "http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/Stats19_Data_2005-2014.zip"
    Traffic_Volume = "https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/498684/tra0201.xls"
    Cycle_Volume = "https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/428727/tra0401.xls"

In [3]:
"""Extracts and the 3 Stats 19 tables and extracts the relevent columns"""

stats19_file = zipfile.ZipFile(io.BytesIO(requests.get(STATS19).content))
Accidents = pd.read_csv(stats19_file.open('Accidents0514.csv'), 
                       index_col = 0, 
                       usecols = [0, 6, 7])
Vehicles = pd.read_csv(stats19_file.open('Vehicles0514.csv'),
                        usecols = [0, 1, 2])
Casualties = pd.read_csv(stats19_file.open('Casualties0514.csv'),
                          usecols = [0, 1, 3, 7])

In [4]:
Traffic_Volume = pd.read_excel(Traffic_Volume, sheet_name = "TRA0201", skiprows = list(range(0,6)), index_col = 0)
Cycle_Volume = pd.read_excel(Cycle_Volume, sheet_name = "TRA0401", skiprows = list(range(0,5)), index_col = 0)
Ten_Year_Volume = {}
for col in list(Traffic_Volume.columns.values)[1:6]:
    Ten_Year_Volume[col] = sum(Traffic_Volume[col][56:66]) #Selected rows refer to 2005 to 2014
Ten_Year_Volume['Pedal Cycles'] = sum(Cycle_Volume['Kilometres'][56:66])

We only care about fatalites so the Casualties and Accidents can both be filtered

In [5]:
Include_Pedestrians = True # If False, the microcides from pedestrans are ignored

Deaths = Casualties[Casualties['Casualty_Severity'] == 1]
if not Include_Pedestrians:
    Deaths = Casualties[Casualties['Casualty_Class'] != 3]
Fatal_Accidents = Accidents[Accidents['Accident_Severity'] == 1]
Accident_Types = {} #This dictionary will take a dictionary of accident types and frequencies for each test

The function below does the blaming and in this version uses the backwards variable to decide on whether to reverse the 'blaming order' which is based on size. The ability to reverse the order allows us to examine the sensitivity of the results to this order.

In [6]:
def Calculate_Microcides(Accident_Type, Number, backwards, caused_fatalities):
    vehicles = Accident_Type[0]
    if len(vehicles) < 2:
        return caused_fatalities
    deaths = Accident_Type[1]
    for i, death in enumerate(deaths):
        if death > 0:
            other_vehicles = vehicles[:i]+vehicles[i+1:]
            if not backwards:
                caused_fatalities[max(other_vehicles)][vehicles[i]] += other_vehicles.count(max(other_vehicles))*death*Number
            else:
                caused_fatalities[min(other_vehicles)][vehicles[i]] += other_vehicles.count(min(other_vehicles))*death*Number
    return caused_fatalities

The function below runs a single test, most of this code can be found in the Traffic_Microcides notebook.

In [7]:
def run_test(name, max_veh, min_veh, deaths, fatal_accidents, vehicles, vehicle_encoding, backwards = False): 
    Fatal_Small_Accidents = fatal_accidents[(fatal_accidents['Number_of_Vehicles'] <= max_veh) & 
                                            (fatal_accidents['Number_of_Vehicles'] >= min_veh)]
    deaths.loc[deaths['Casualty_Class'] == 3, 'Vehicle_Reference'] = 0 # Removes pedestrians vehicle
    Grouped_Deaths = deaths.groupby(by = [deaths.columns[0], 'Vehicle_Reference']).sum()
    Grouped_Deaths.drop('Casualty_Class', axis=1, inplace=True)
    Grouped_Deaths.columns = ['Fatalities']
    Fatality_Vehicles = Fatal_Small_Accidents.merge(vehicles, how = 'left', left_index = 
                                          True, right_on = deaths.columns[0])
    Fatality_Vehicles = Fatality_Vehicles[[deaths.columns[0], 'Vehicle_Reference', 'Vehicle_Type']] 
    All_Microcide_Vehicles = Fatality_Vehicles.merge(Grouped_Deaths, how = 'outer', 
                                                 left_on = [deaths.columns[0],'Vehicle_Reference'],
                                                 right_index = True)
    All_Microcide_Vehicles.fillna(0, inplace = True) 
    All_Microcide_Vehicles['Vehicle_Group'] = 0
    for k, v in vehicle_encoding.items():
        All_Microcide_Vehicles.loc[All_Microcide_Vehicles['Vehicle_Type'].isin(v), 'Vehicle_Group'] = k
    All_Microcide_Vehicles.sort_values(by = [deaths.columns[0], 'Vehicle_Group', 'Fatalities'], inplace = True)
    Accident_Group = All_Microcide_Vehicles.groupby(deaths.columns[0]).agg(lambda x: tuple(x))
    Accident_Group['Accident_Type'] = Accident_Group[['Vehicle_Group', 'Fatalities']].apply(tuple, axis = 1)
    All_Accident_Types = Accident_Group[['Accident_Type']]
    Accident_Type_Counts = All_Accident_Types['Accident_Type'].value_counts()
    Accident_Types[name] = Accident_Type_Counts
    Caused_Fatalities = {} # AAccident_Type_Counts dictionary of deaths by vehicle. The keys are the vehicle types and values are running totals.
    for n in range(0,10):
            Caused_Fatalities[n] = {}
            for m in range(0,10):
                Caused_Fatalities[n][m] = 0
    for acc_type, count in Accident_Type_Counts.iteritems():
        Caused_Fatalities = Calculate_Microcides(acc_type, count, backwards, Caused_Fatalities)
    return Caused_Fatalities

The vehicle types are reassigned based on the vehicle types used by the DfT in their traffic volume and road death tables.
The number codes used also double as a sorting mechanism where by 'smaller' vehicles are assigned lower numbers.

In [8]:
Vehicle_Encoding = {
1: (16, 22),             # Horses and Mobility Scooters
2: (1,),                 # Pedal Cycle
3: (2, 3, 4 ,5, 23, 97), # Mopeds and Motorcycle 
4: (9, 8, 19, 20),       # Cars and taxis
5: (19,),                # Vans 
6: (17,),                # Agricultural Vehicles
7: (18,),                # Trams
8: (10, 11),             # Buses and Coaches
9: (20, 21, 98)          # HGVs
}

The column names in the traffic files and vehicle codes described by the Vehicle_Encoding dictionary are mapped using the dictionary below.

In [9]:
Volume_Accident_Coding = {
'Buses & Coaches':  8,
'Cars and taxis':   4,
'Goods vehicles 2': 9,
'Light\nvans 1':    5,
'Motorcycles':      3,
'Pedal Cycles':     2
}
Full_Results = {} #This dictionary will take a dataframe for each test that show the attributed deaths for each vehicle

The function below runs each test within a dictionary of tests and returns the attributed deaths for each vehicle type

In [10]:
def run_all_tests(test_settings):
    Microcides = pd.DataFrame.from_dict(Ten_Year_Volume, orient = 'index')
    Microcides.columns = ['Traffic Volume']
    for name, setting in test_settings.items():
        Microcides[name] = 0
        test_results = run_test(name, *setting)
        Full_Results[name] = pd.DataFrame.from_dict(test_results, orient = 'index')
        for k, v, in Volume_Accident_Coding.items():
            Microcides = Microcides.set_value(k, name, sum(test_results[v].values()))
    return Microcides

Each element in the dictionary below defines the settings for a test. The first 2 numbers are the max and min accident sizes respectively and the boolean determines whether the blaming order should be reversed.

In [11]:
test_settings = {
"Default": (4, 0, Deaths, Fatal_Accidents, Vehicles, Vehicle_Encoding, False),
"Backwards": (4, 0, Deaths, Fatal_Accidents, Vehicles, Vehicle_Encoding, True),
"All": (40, 0, Deaths, Fatal_Accidents, Vehicles, Vehicle_Encoding, False)
}

In [12]:
All_Microcides = run_all_tests(test_settings)
All_Microcides

C:\Miniconda3\envs\jollydata\lib\site-packages\pandas\core\indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Traffic Volume,All,Backwards,Default
Pedal Cycles,47.2,50,104,50
Goods vehicles 2,269.8,3608,2446,3163
Motorcycles,48.6,363,606,363
Cars and taxis,3916.6,13936,14265,13267
Light\nvans 1,665.8,1598,1179,1526
Buses & Coaches,48.9,919,753,894


In [13]:
"""Each of the elements in this dictionary is the results of a test. The rows refer to the 'perpetrator' vehicle, with
the columns being a breakdown of the types of victim.
See cell 8 for the key"""

Full_Results['Default']

,0,1,2,3,4,5,6,7,8,9
0,174,1,26,49,54,3,0,0,2,1
1,0,0,0,0,0,0,0,0,0,0
2,30,0,8,5,3,0,0,0,2,2
3,156,2,19,155,31,0,0,0,0,0
4,4089,10,717,2854,5404,95,9,0,46,43
5,387,2,86,341,672,31,0,0,1,6
6,9,1,7,105,87,6,1,0,0,6
7,7,0,1,0,0,0,0,0,1,0
8,451,0,54,92,278,14,0,0,2,3
9,726,3,234,374,1493,134,8,0,15,176


The function below takes a vehicle code (currently set to pedal cycles) and prints counts of all the accident types. This is primarily for reality checking

In [14]:
def show_accident_types(vehicle_type, test_name):
    for k,v in Accident_Types[test_name].items():
        if vehicle_type in k[0]:
            print(k,v)
show_accident_types(2, 'All')

((2, 4), (1.0, 0.0)) 557
((2, 9), (1.0, 0.0)) 220
((2,), (1.0,)) 128
((2, 5), (1.0, 0.0)) 70
((2, 8), (1.0, 0.0)) 48
((2, 4, 4), (1.0, 0.0, 0.0)) 46
((0, 2), (1.0, 0.0)) 30
((0, 2), (0.0, 1.0)) 26
((2, 2, 4), (0.0, 1.0, 0.0)) 22
((2, 3), (1.0, 0.0)) 11
((2, 4, 4, 4), (1.0, 0.0, 0.0, 0.0)) 10
((2, 4, 5), (1.0, 0.0, 0.0)) 7
((2, 2), (0.0, 1.0)) 6
((2, 6), (1.0, 0.0)) 6
((2, 4, 9), (1.0, 0.0, 0.0)) 5
((2, 3), (1.0, 1.0)) 4
((0, 2, 8), (1.0, 0.0, 0.0)) 4
((2, 4, 8), (1.0, 0.0, 0.0)) 3
((2, 3, 4), (0.0, 1.0, 0.0)) 3
((2, 4, 4), (0.0, 0.0, 1.0)) 2
((2, 3, 4, 4), (0.0, 1.0, 0.0, 0.0)) 2
((2, 9), (0.0, 1.0)) 2
((2, 4), (0.0, 1.0)) 2
((2, 5, 5), (1.0, 0.0, 0.0)) 2
((2, 8), (0.0, 1.0)) 2
((2, 9, 9), (1.0, 0.0, 0.0)) 2
((2, 4, 4, 5), (1.0, 0.0, 0.0, 0.0)) 2
((2, 2, 9), (0.0, 1.0, 0.0)) 2
((2, 4, 5, 9, 9), (1.0, 0.0, 0.0, 0.0, 0.0)) 1
((2, 4, 5, 5), (1.0, 0.0, 0.0, 0.0)) 1
((2, 2, 4, 4), (0.0, 1.0, 0.0, 0.0)) 1
((2, 2, 4, 5), (0.0, 1.0, 0.0, 0.0)) 1
((2, 4, 4, 4, 4, 4), (1.0, 0.0, 0.0, 0.0, 0.0, 0

The function below takes the counts of accident types from a test and summarises them in a dataframe

In [15]:
def explore_accident_types(test_name):
    Accident_Type_Exploration = pd.DataFrame(Accident_Types[test_name])
    Accident_Type_Exploration.columns = ['Accident_Count']
    Accident_Type_Exploration['Type'] = Accident_Types[test_name].index
    Accident_Type_Exploration['Vehicles'] = Accident_Type_Exploration['Type'].apply(lambda x: x[0]) 
    Accident_Type_Exploration['Deaths'] = Accident_Type_Exploration['Type'].apply(lambda x: x[1])
    Accident_Type_Exploration['Vehicle_Count'] = Accident_Type_Exploration['Vehicles'].apply(len)
    Accident_Type_Exploration['Deaths_Per_Accident'] = Accident_Type_Exploration['Deaths'].apply(sum)
    Accident_Type_Exploration['Total_Deaths'] = Accident_Type_Exploration['Accident_Count'] * \
                                            Accident_Type_Exploration['Deaths_Per_Accident']
    Accident_Type_Exploration[['Deaths_Per_Accident','Total_Deaths','Accident_Count']].groupby('Deaths_Per_Accident').sum()
    return Accident_Type_Exploration

The function below takes the above summarised dataframe and examines the cumulative counts and percentages as you increase the number of vehicles included.

In [16]:
def get_accident_size_table(test_name):
    Accident_Types_Exp = explore_accident_types(test_name)
    Accident_Sizes = Accident_Types_Exp[['Vehicle_Count', 'Accident_Count', 'Total_Deaths']].groupby('Vehicle_Count').sum()
    Accident_Sizes['Total_Vehicles_Involved'] = Accident_Sizes['Accident_Count']*Accident_Sizes.index
    Accident_Sizes['Percentage_Deaths'] = Accident_Sizes['Total_Deaths']/sum(Accident_Sizes['Total_Deaths'])*100
    Accident_Sizes['Percentage_Accidents'] = Accident_Sizes['Accident_Count']/sum(Accident_Sizes['Accident_Count'])*100
    Accident_Sizes['Cumulative_%_Vehicles'] = 100*Accident_Sizes['Total_Vehicles_Involved'].cumsum()/ \
                                              Accident_Sizes['Total_Vehicles_Involved'].sum()
    Accident_Sizes['Cumulative_%_Accidents'] = 100*Accident_Sizes['Accident_Count'].cumsum()/Accident_Sizes['Accident_Count'].sum()
    return Accident_Sizes

In [17]:
explore_accident_types('All')

,Accident_Count,Type,Vehicles,Deaths,Vehicle_Count,Deaths_Per_Accident,Total_Deaths
"((4,), (1.0,))",3410,"((4,), (1.0,))","(4,)","(1.0,)",1,1,3410
"((0, 4), (1.0, 0.0))",2939,"((0, 4), (1.0, 0.0))","(0, 4)","(1.0, 0.0)",2,1,2939
"((4, 4), (0.0, 1.0))",2295,"((4, 4), (0.0, 1.0))","(4, 4)","(0.0, 1.0)",2,1,2295
"((3, 4), (1.0, 0.0))",1686,"((3, 4), (1.0, 0.0))","(3, 4)","(1.0, 0.0)",2,1,1686
"((3,), (1.0,))",991,"((3,), (1.0,))","(3,)","(1.0,)",1,1,991
"((4, 9), (1.0, 0.0))",783,"((4, 9), (1.0, 0.0))","(4, 9)","(1.0, 0.0)",2,1,783
"((4, 4, 4), (0.0, 0.0, 1.0))",584,"((4, 4, 4), (0.0, 0.0, 1.0))","(4, 4, 4)","(0.0, 0.0, 1.0)",3,1,584
"((2, 4), (1.0, 0.0))",557,"((2, 4), (1.0, 0.0))","(2, 4)","(1.0, 0.0)",2,1,557
"((0, 9), (1.0, 0.0))",548,"((0, 9), (1.0, 0.0))","(0, 9)","(1.0, 0.0)",2,1,548
"((0, 8), (1.0, 0.0))",367,"((0, 8), (1.0, 0.0))","(0, 8)","(1.0, 0.0)",2,1,367


In [18]:
get_accident_size_table('All')

,Accident_Count,Total_Deaths,Total_Vehicles_Involved,Percentage_Deaths,Percentage_Accidents,Cumulative_%_Vehicles,Cumulative_%_Accidents
Vehicle_Count,,,,,,,
1,5282,5713,5282,24.761616,24.703021,12.225437,24.703021
2,12377,13171,24754,57.086512,57.885137,69.519732,82.588158
3,2584,2880,7752,12.482663,12.084931,87.462099,94.673090
4,741,824,2964,3.571429,3.465532,94.322416,98.138621
5,214,245,1070,1.061893,1.000842,96.798982,99.139463
6,90,111,540,0.481103,0.420915,98.048837,99.560378
7,38,50,266,0.216713,0.177720,98.664506,99.738097
8,20,22,160,0.095354,0.093537,99.034834,99.831634
9,16,23,144,0.099688,0.074829,99.368129,99.906463


### Key Exploratory Results and Conclusions

* Reversing the order of vehicles (the 'Backwards' test) had a limited impact on the attributed deaths for most vehicles, the exceptions being pedal cycles and motorcycles
* Most (>95%) accidents and deaths occur in accidents with 4 or fewer vehicles.
* Including large accidents has a minimal effect on the attributed deaths compared to setting the maximum size to 4 vehicles. The largest impact would be on goods vehicles would increase attributed deaths by around 15%.